# Cooling Effect Module Draft 

In [18]:
# Install the GUS framework.

import sys

!{sys.executable} -m pip install --index-url https://test.pypi.org/simple/ --extra-index-url https://pypi.org/simple gus==0.1.7

Looking in indexes: https://test.pypi.org/simple/, https://pypi.org/simple


In [19]:
import numpy as np
import pandas as pd

In [20]:
DBH = 15
TH = 5
TCD = 4
TCDth = 3

In [21]:
def compute_under_canopy_area(crown_diameter):
    return pow((crown_diameter / 2), 2) * np.pi


def compute_leaf_area_index(
    dbh,
    tree_height,
    crown_depth,
    crown_diameter,
    crown_missing=0,
    shade_factor=0.85,
):
    """The function given allometrics of a tree computes its leaf, bark and plant area indices.

    Args:
        dbh: (:obj:`float`): the diameter in cm of the trunk usually measured at 1.3m from the ground.
        tree_height: (:obj:`float`): The tree height in meters.
        crown_debth: (:obj:`float`): The vertical length of tree crown in meters.
        crown_diameter: (:obj:`float`): The horizontal length (diameter) of tree crown in meters.
        crown_missing: (:obj:`float`): The percentage loss of the crown.
        shade_factor: (:obj:`float`): The percentage of sky covered by foliage and branches.

        Shade factor, is the percentage of sky covered by foliage and branches within the perimeter of individual tree crowns,
        can vary by species from about 60% to 95% when trees are in-leaf (McPherson, 1984). The value below is set according to
        Glasgow mean and needs to be updated per city or per tree.

    Returns:
        (:obj:`float`): It eturns the LAI.
    Note:
        The beta multipliers and the main equation is based on Nowak (1996).

    TODO:
        Parametrize beta multipliers.
    """
    loss = crown_missing
    th = tree_height
    cw = crown_diameter
    cd = crown_depth
    sf = shade_factor
    beta_0 = -4.3309
    beta_1 = 0.2942
    beta_2 = 0.7312
    beta_3 = 5.7217
    beta_4 = 0.0148

    # Outer surface area estimate below is based on Gacka-Grzesikiewicz (1980).
    under_canopy = compute_under_canopy_area(cw)
    crown_surface = np.pi * cw * (cd + cw) / 2
    leaf_area = (1 - loss) * np.exp(
        beta_0 + beta_1 * th + beta_2 * cw + beta_3 * sf - beta_4 * crown_surface
    )
    leaf_area_index = leaf_area / under_canopy  # m^2/m^2
    return leaf_area_index  # An intuitive way to think about LAI is as the one-sided green leaf area per unit ground surface area. !!

In [22]:
compute_leaf_area_index(dbh=DBH, tree_height=TH, crown_diameter=TCD, crown_depth=TCDth)

5.733824472020663

In [23]:
def compute_gross_canopy_index(tree_height, crown_depth, crown_diameter):
    """The function given allometrics of a tree computes an index to account shading effect immediately underneath its canopy.

    Args:
        tree_height: (:obj:`float`): The tree height in meters.
        crown_height: (:obj:`float`): The vertical length of tree crown in meters.
        crown_width: (:obj:`float`): The horizontal length (diameter) of tree crown in meters.

    Returns:
        (:obj:`float`): returns the GCI index
    Note:
        The index is based on (Zhang et.al., 2020). It accounts for the overall immediate shading effect of a tree.
        It accounts both crown diameter (tw) and the crown base height (th - ch), the distance from the ground to the start of
        the canopy.
    """
    th = tree_height
    cw = crown_diameter
    cd = crown_depth
    crown_base_height = th - cd
    if crown_base_height < 0.1:
        crown_base_height = 0.1
    GCI = cw / crown_base_height
    return GCI  # GCI is just a ratio of crown diameter to crown base height. It is a measure of the overall immediate shading effect of a tree.

In [24]:
compute_gross_canopy_index(tree_height=TH, crown_depth=TCDth, crown_diameter=TCD)

2.0

In [25]:
def compute_crown_volume_index(leaf_area_index, crown_canopy_index):
    """The function given computes a composite index taking into account both leaf density and canopy shape.

    Args:
        leaf_area_indext: (:obj:`float`): The index on the leaf density per the area under the canopy.
        crown_canopy_index: (:obj:`float`): The index that accounts for the geomtric shape of the canopy relevant to shading.

    Returns:
        (:obj:`float`): returns the CVI index
    Note:
        The index is adopted from (Zhang et.al., 2020).
    """
    # CVI is a composite index taking into account both leaf density and canopy shape.
    return leaf_area_index * crown_canopy_index

In [26]:
LAI = compute_leaf_area_index(DBH, TH, TCD, TCDth)
GCI = compute_gross_canopy_index(TH, TCDth, TCD)
compute_crown_volume_index(LAI, GCI)

11.547033684456904

In [27]:
df = pd.read_csv("trees_yearly.csv")
df.set_index("AgentID", inplace=True)
df.rename(columns={"Step": "year"}, inplace=True)
df.tail(20)

,year,species,dbh,height,crownH,crownW,canopy_overlap,cle,condition,dieback,biomass,seq,carbon,deroot,detrunk,mulched,burnt,coordinates
AgentID,,,,,,,,,,,,,,,,,,
153,50,decidu,30.439456,15.931190,9.125725,8.037718,0.139470,0.939746,excellent,0.000985,515.684843,20.717222,257.842422,0.0,0.0,0.012695,0.000000,"(6, 9)"
154,50,decidu,27.691566,15.133120,8.613116,7.557668,0.377894,0.844516,excellent,0.003000,416.796679,20.405725,208.398339,0.0,0.0,0.031260,0.000000,"(9, 8)"
155,50,decidu,24.806399,14.255167,8.037429,7.027223,0.509405,0.775526,excellent,0.002000,325.387001,15.288357,162.693500,0.0,0.0,0.000000,0.016269,"(9, 2)"
156,50,decidu,24.952244,14.300632,8.067451,7.054686,0.373618,0.831053,good,0.010953,329.707622,16.330511,164.853811,0.0,0.0,0.090280,0.000000,"(10, 4)"
157,50,decidu,24.461331,14.147108,7.966007,6.961970,0.483045,0.788265,excellent,0.001567,315.290348,15.263301,157.645174,0.0,0.0,0.012351,0.000000,"(4, 7)"
158,50,conifer,15.256274,9.171866,5.349862,3.472220,0.243380,0.873386,replaced,1.000000,98.620274,0.000000,49.310137,0.0,0.0,0.000000,0.000000,"(1, 5)"
159,50,decidu,19.784401,12.606888,6.943822,6.039380,0.696196,0.682728,excellent,0.004747,195.585539,10.087746,97.792769,0.0,0.0,0.023213,0.000000,"(3, 6)"
160,50,decidu,17.302682,11.721643,6.360461,5.520953,0.182742,0.917570,replaced,1.000000,144.661371,0.000000,72.330685,0.0,0.0,0.000000,0.000000,"(2, 10)"
161,50,conifer,15.965977,9.535299,5.520800,3.596589,0.114319,0.940309,excellent,0.007302,110.221784,5.618035,55.110892,0.0,0.0,0.020122,0.000000,"(13, 8)"


In [28]:
tree_42 = df.xs(42)
tree_31 = df.xs(31)
tree_31.tail()

,year,species,dbh,height,crownH,crownW,canopy_overlap,cle,condition,dieback,biomass,seq,carbon,deroot,detrunk,mulched,burnt,coordinates
AgentID,,,,,,,,,,,,,,,,,,
31,46,decidu,49.904146,20.838819,11.759976,10.735481,0.346223,0.878366,excellent,0.003356,1568.577591,12.673916,784.288796,0.0,0.0,0.131604,0.000000,"(3, 9)"
31,47,decidu,50.255590,20.918409,11.791482,10.772873,0.348965,0.877084,excellent,0.004356,1593.544189,12.483299,796.772094,0.0,0.0,0.173537,0.000000,"(3, 9)"
31,48,decidu,50.608610,20.998100,11.822557,10.810027,0.349508,0.877037,excellent,0.005356,1618.843506,12.649659,809.421753,0.0,0.0,0.216764,0.000000,"(3, 9)"
31,49,decidu,50.947728,21.074413,11.851866,10.845337,0.350415,0.876797,excellent,0.004356,1643.355149,12.255822,821.677575,0.0,0.0,0.178962,0.000000,"(3, 9)"
31,50,decidu,51.242808,21.140629,11.876938,10.875758,0.352722,0.875722,excellent,0.005356,1664.850374,10.747613,832.425187,0.0,0.0,0.000000,0.222924,"(3, 9)"


In [29]:
tree_42 = tree_42[["year", "species", "dbh", "height", "crownH", "crownW"]]
tree_31 = tree_31[["year", "species", "dbh", "height", "crownH", "crownW"]]

In [33]:
tree_31["LAI"] = tree_31.apply(
    lambda row: compute_leaf_area_index(
        row["dbh"], row["height"], row["crownH"], row["crownW"]
    ),
    axis=1,
)
tree_31["GPI"] = tree_31.apply(
    lambda row: compute_gross_canopy_index(row["height"], row["crownH"], row["crownW"]),
    axis=1,
)
tree_31["CVI"] = tree_31.apply(
    lambda row: compute_crown_volume_index(row["LAI"], row["GPI"]), axis=1
)
tree_31.head()

,year,species,dbh,height,crownH,crownW,LAI,GPI,CVI
AgentID,,,,,,,,,
31,1,decidu,10.160808,8.778266,4.523209,3.917328,15.203610,0.920629,13.996883
31,2,decidu,11.624329,9.443938,4.918861,4.259451,17.459500,0.941299,16.434609
31,3,decidu,13.061136,10.061116,5.297682,4.588556,19.836204,0.963288,19.107969
31,4,decidu,14.468776,10.636331,5.659583,4.904473,22.302067,0.985478,21.978188
31,5,decidu,15.966213,11.220819,6.034541,5.233474,25.050609,1.009100,25.278572


In [34]:
def compute_change(current_value, initial_value):
    """The function to copmpute percentage change with respect to a given base value."""
    if initial_value == 0:
        return 0

    return ((current_value - initial_value) / initial_value) * 100

In [35]:
base_lai = list(tree_31["LAI"])[
    0
]  # The first value of the LAI column is the base value. year 0
base_cvi = list(tree_31["CVI"])[0]
base_gpi = list(tree_31["GPI"])[0]
tree_31["dLAI"] = tree_31["LAI"].apply(lambda row: compute_change(row, base_lai))
tree_31["dGPI"] = tree_31["GPI"].apply(lambda row: compute_change(row, base_gpi))
tree_31["dCVI"] = tree_31["CVI"].apply(lambda row: compute_change(row, base_cvi))

In [36]:
tree_31.head(25)

,year,species,dbh,height,crownH,crownW,LAI,GPI,CVI,dLAI,dGPI,dCVI
AgentID,,,,,,,,,,,,
31,1,decidu,10.160808,8.778266,4.523209,3.917328,15.203610,0.920629,13.996883,0.000000,0.000000,0.000000
31,2,decidu,11.624329,9.443938,4.918861,4.259451,17.459500,0.941299,16.434609,14.837861,2.245205,17.416206
31,3,decidu,13.061136,10.061116,5.297682,4.588556,19.836204,0.963288,19.107969,30.470362,4.633638,36.515886
31,4,decidu,14.468776,10.636331,5.659583,4.904473,22.302067,0.985478,21.978188,46.689292,7.043948,57.022009
31,5,decidu,15.966213,11.220819,6.034541,5.233474,25.050609,1.009100,25.278572,64.767510,9.609858,80.601433
31,6,decidu,17.413251,11.762272,6.387056,5.544477,27.803181,1.031489,28.678677,82.872240,12.041775,104.893303
31,7,decidu,18.853117,12.280991,6.728240,5.847183,30.608847,1.053024,32.231863,101.326182,14.380978,130.278857
31,8,decidu,20.229373,12.760126,7.045416,6.130217,33.327437,1.072708,35.750623,119.207401,16.519083,155.418453
31,9,decidu,21.695662,13.254491,7.373732,6.424993,36.236696,1.092545,39.590216,138.342716,18.673742,182.850220


## Cooling Effect

### Calibration

In [37]:
dT_max = 0.67
m1 = 0.1
m2 = 0.75
m4 = 0.75
m5 = 0.2
T0 = 10
T1 = 18
T2 = 26
T3 = 35
T4 = 45
T5 = 50
avg_conifer_cooling_factor = 0.66


def cooling_multiplier(T):
    if T < T0:
        return 0
    if T < T1:
        return m1 * dT_max
    if T < T2:
        return m2 * dT_max
    if T < T3:
        return dT_max
    if T < T4:
        return m4 * dT_max
    if T < T5:
        return m5 * dT_max
    return 0


# Comparing this index of the canopy, and how it changes across a timespan from A to B
# We can estimate the further cooling potential of that tree canopy in B relative to A (in degrees C)
def compute_cooling(canopy_change_cvi, T=30, species=None):
    # The CVI is a composite index taking into account both leaf density and canopy shape. The higher the CVI the greater the cooling effect.
    if species == "conifer":
        species_factor = 0.6
    elif species == "decidu":
        species_factor = 0.9
    else:
        species_factor = 1
    return species_factor * np.sqrt(canopy_change_cvi / 10) * cooling_multiplier(T)

In [38]:
cooling_multiplier(30)

0.67

In [39]:
grid_temperature = 10
tree_31["dT"] = tree_31.apply(
    lambda row: compute_cooling(
        row["dCVI"], T=grid_temperature, species=row["species"]
    ),
    axis=1,
)
tree_31.tail()

,year,species,dbh,height,crownH,crownW,LAI,GPI,CVI,dLAI,dGPI,dCVI,dT
AgentID,,,,,,,,,,,,,
31,46,decidu,49.904146,20.838819,11.759976,10.735481,80.901747,1.182472,95.664082,432.121973,28.441797,583.467022,0.460602
31,47,decidu,50.255590,20.918409,11.791482,10.772873,81.469421,1.180340,96.161584,435.855785,28.210130,587.021400,0.462002
31,48,decidu,50.608610,20.998100,11.822557,10.810027,82.049072,1.178135,96.664875,439.668372,27.970652,590.617134,0.463415
31,49,decidu,50.947728,21.074413,11.851866,10.845337,82.615258,1.175959,97.152155,443.392397,27.734302,594.098483,0.464779
31,50,decidu,51.242808,21.140629,11.876938,10.875758,83.115740,1.174020,97.579541,446.684262,27.523686,597.151919,0.465972


In [40]:
grid_temperature = 34
tree_31["dT"] = tree_31.apply(
    lambda row: compute_cooling(
        row["dCVI"], T=grid_temperature, species=row["species"]
    ),
    axis=1,
)
tree_31.tail()

,year,species,dbh,height,crownH,crownW,LAI,GPI,CVI,dLAI,dGPI,dCVI,dT
AgentID,,,,,,,,,,,,,
31,46,decidu,49.904146,20.838819,11.759976,10.735481,80.901747,1.182472,95.664082,432.121973,28.441797,583.467022,4.606016
31,47,decidu,50.255590,20.918409,11.791482,10.772873,81.469421,1.180340,96.161584,435.855785,28.210130,587.021400,4.620025
31,48,decidu,50.608610,20.998100,11.822557,10.810027,82.049072,1.178135,96.664875,439.668372,27.970652,590.617134,4.634153
31,49,decidu,50.947728,21.074413,11.851866,10.845337,82.615258,1.175959,97.152155,443.392397,27.734302,594.098483,4.647790
31,50,decidu,51.242808,21.140629,11.876938,10.875758,83.115740,1.174020,97.579541,446.684262,27.523686,597.151919,4.659719


In [41]:
grid_temperature = 48
tree_31["dT"] = tree_31.apply(
    lambda row: compute_cooling(
        row["dCVI"], T=grid_temperature, species=row["species"]
    ),
    axis=1,
)
tree_31.tail()

,year,species,dbh,height,crownH,crownW,LAI,GPI,CVI,dLAI,dGPI,dCVI,dT
AgentID,,,,,,,,,,,,,
31,46,decidu,49.904146,20.838819,11.759976,10.735481,80.901747,1.182472,95.664082,432.121973,28.441797,583.467022,0.921203
31,47,decidu,50.255590,20.918409,11.791482,10.772873,81.469421,1.180340,96.161584,435.855785,28.210130,587.021400,0.924005
31,48,decidu,50.608610,20.998100,11.822557,10.810027,82.049072,1.178135,96.664875,439.668372,27.970652,590.617134,0.926831
31,49,decidu,50.947728,21.074413,11.851866,10.845337,82.615258,1.175959,97.152155,443.392397,27.734302,594.098483,0.929558
31,50,decidu,51.242808,21.140629,11.876938,10.875758,83.115740,1.174020,97.579541,446.684262,27.523686,597.151919,0.931944
